# Create Embeddings Table

In [ ]:
# conda install -c huggingface transformers
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration, AutoTokenizer, TFBertModel, AutoModel, BertTokenizer

# Tokenizers and Models

In [ ]:
# bio_bert_model = AutoModel.from_pretrained("gsarti/biobert-nli")
# bio_bert_tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
# original_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# T5Abstract_model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
T5tokens = T5Tokenizer.from_pretrained('t5-base')

In [ ]:
from transformers import BioGptModel, BioGptConfig, BioGptTokenizer
biogpttokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
biogptmodel = BioGptModel.from_pretrained("microsoft/biogpt")

# Get Abstracts

In [ ]:
import os
import yaml
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# set up
with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)
    
bucket_path = os.path.join(config['paths']['root'], config['paths']['mount'])


In [ ]:
# Create engine to connect to database
engine = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

In [10]:
# pull all the abstracts
query = ''' SELECT * FROM abstracts'''
with engine.connect() as conn: 
    results = conn.execute(query)

In [ ]:
abstracts = pd.DataFrame(results.fetchall())
print(abstracts.shape)
abstracts = abstracts.dropna(how = 'all', subset='abstract').drop('index', axis = 1).reset_index(drop = True)
print(abstracts.shape)